In [ ]:
import torch
import numpy as np
import os
import sys

# 将当前目录添加到系统路径，确保能导入 utils
# 假设 ipynb 文件在 VAMoE-main/vamoe 目录下
sys.path.append(os.getcwd())

from utils.data_loader_npyfiles import Era5Data

data_params = {
    'name': 'era5',
    'root_dir': './data',
    'feature_dims': 65,   # 高空变量数 (例如 5个变量 * 13层 = 65)
    't_in': 1,            # 输入时间步
    't_out_train': 1,     # 训练输出步数
    't_out_valid': 1,
    't_out_test': 20,
    'valid_interval': 1,
    'test_interval': 1,
    'train_interval': 1,
    'pred_lead_time': 6,
    'data_frequency': 1,
    'train_period': [2025, 2025], # <--- 【重要】请修改为你数据文件夹中存在的年份
    'valid_period': [2025, 2025],
    'test_period': [2025, 2025],
    'patch': True,
    'patch_size': 4,
    'batch_size': 1,
    'num_data_workers': 0,
    'grid_resolution': 1.4,
    'h_size': 256,        # 图片高度
    'w_size': 512,        # 图片宽度
    'surface_feature_size': 0, # 表面变量数量
    'pressure_level_num': 13,
}

print("=== 1. 初始化 Dataset ===")
try:
    dataset = Era5Data(data_params, run_mode='train')
    print(f"Dataset 初始化成功。总样本数: {len(dataset)}")
except Exception as e:
    print(f"Dataset 初始化失败 (可能是路径或统计文件缺失): {e}")
    # 如果初始化失败，无法继续后续步骤，停止运行
    dataset = None

if dataset:
    # 2. 最终输出测试 (__getitem__)
    print("\n=== 2. 检查 __getitem__ 最终输出 ===")
    try:
        # 获取第 0 个样本
        idx = 0
        inputs, labels = dataset[idx]
        
        print(f"最终 Inputs Shape: {inputs.shape}")
        print(f"最终 Labels Shape: {labels.shape}")
        print(f"数据类型: {inputs.dtype}")
        print(f"数据范围: Min={np.min(inputs):.3f}, Max={np.max(inputs):.3f}")
    except Exception as e:
        print(f"__getitem__ 执行报错: {e}")

    # 3. 逐步拆解内部流程 (Debug 维度变化)
    print("\n=== 3. 逐步拆解内部处理流程 (Debug 维度) ===")
    
    # 3.1 模拟获取文件路径
    from utils.data_loader_npyfiles import get_datapath_from_date
    cur_input_data_idx = 0
    half_path, date_time = get_datapath_from_date(dataset.start_date, cur_input_data_idx)
    print(f"步骤 [1] 文件路径前缀: {half_path}")
    
    try:
        # 3.2 读取原始数据 (_get_weather_data)
        x_raw, x_surf_raw = dataset._get_weather_data(half_path)
        print(f"步骤 [2] _get_weather_data 返回 (单帧数据):")
        print(f"   高空变量 Shape: {x_raw.shape} (预期: [H, W, Level, Feature])")
        if x_surf_raw is not None:
            print(f"   表面变量 Shape: {x_surf_raw.shape}")
        else:
            print("   地表面变量: None")
        
        # 3.3 归一化 (_normalize)
        x_norm, x_surf_norm = dataset._normalize(x_raw, x_surf_raw)
        print(f"步骤 [3] _normalize 返回 Shape: {x_norm.shape}")
        
        # 3.4 模拟时间步堆叠 (Stack vs Concatenate)
        print(f"步骤 [4] 模拟多时间步合并 (t_in={dataset.t_in}):")
        inputs_lst = [x_norm] * dataset.t_in
        
        # 模拟代码中原本的 concatenate
        x_concat = np.concatenate(inputs_lst, axis=0).astype(np.float32)
        print(f"   -> 使用 np.concatenate 后 x 的 Shape: {x_concat.shape} (注意第0维)")
        
        # 模拟修复建议的 stack
        x_stack = np.stack(inputs_lst, axis=0).astype(np.float32)
        print(f"   -> 使用 np.stack 后 x 的 Shape:       {x_stack.shape} (注意第0维)")
        
        # 3.5 进入 _process_fn 处理
        print(f"步骤 [5] 进入 _process_fn 处理:")
        
        # 构造 Dummy Label (仅用于测试形状)
        label_dummy = x_concat 
        x_surf_dummy = None 
        
        print("   尝试传入 Concatenate 的结果 (4维数据)...")
        try:  
        
            # 调用 _process_fn
            fin_in, fin_out = dataset._process_fn(x_concat, x_surf_dummy, label_dummy, x_surf_dummy)
            print(f"   -> _process_fn 处理成功!")
            print(f"   -> 结果 Inputs Shape: {fin_in.shape}")
            # 如果这里成功，说明你添加的 `if len(x.shape) == 4` 补丁生效了
            if len(x_concat.shape) == 4:
                print("   [结论] 你的 4维->5维 补丁代码正在发挥作用。")
        except Exception as e:
            print(f"   -> _process_fn 报错: {e}")
            print("   [结论] 如果报错信息是 'expected 5 dimensions'，说明补丁未生效或未被执行。")

    except FileNotFoundError:
        print("错误：找不到数据文件，无法演示内部流程。请检查 data_params['root_dir']。")
    except Exception as e:
        print(f"内部流程测试出错: {e}")

=== 1. 初始化 Dataset ===
Dataset 初始化成功。总样本数: 732

=== 2. 检查 __getitem__ 最终输出 ===
最终 Inputs Shape: (65, 256, 512)
最终 Labels Shape: (65, 256, 512)
数据类型: float32
数据范围: Min=-64.475, Max=2701.143

=== 3. 逐步拆解内部处理流程 (Debug 维度) ===
步骤 [1] 文件路径前缀: 2025/2025-01-01/00-00-00-
步骤 [2] _get_weather_data 返回 (单帧数据):
   高空变量 Shape: (721, 1440, 13, 5) (预期: [H, W, Level, Feature])
   表面变量: None
步骤 [3] _normalize 返回 Shape: (721, 1440, 13, 5)
步骤 [4] 模拟多时间步合并 (t_in=1):
   -> 使用 np.concatenate 后 x 的 Shape: (721, 1440, 13, 5) (注意第0维)
   -> 使用 np.stack 后 x 的 Shape:       (1, 721, 1440, 13, 5) (注意第0维)
步骤 [5] 进入 _process_fn 处理:
   尝试传入 Concatenate 的结果 (4维数据)...
   -> _process_fn 处理成功!
   -> 结果 Inputs Shape: (65, 256, 512)
   [结论] 你的 4维->5维 补丁代码正在发挥作用。
